In [1]:
from flask import Response
from flask import Flask
from flask import render_template
import threading
import datetime
import cv2
import numpy as np
from imutils.object_detection import non_max_suppression

In [2]:
hog = cv2.HOGDescriptor() 
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector()) 

In [3]:
app = Flask(__name__)

In [4]:
@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html')

In [5]:
vs = cv2.VideoCapture('http://127.0.0.1:8080/')

In [6]:
outputFrame = None
lock = threading.Lock()

In [7]:
def generate():
    
    global outputFrame, lock
    
    while True:
        with lock:
            if outputFrame is None:
                continue
                
            timestamp = datetime.datetime.now()
            cv2.putText(outputFrame, timestamp.strftime(
                "%A %d %B %Y %I:%M:%S%p"), (10, outputFrame.shape[0] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)

            (flag, encodedImage) = cv2.imencode(".jpg", outputFrame)
            outputFrame = None

            if not flag:
                continue

        yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' +
               bytearray(encodedImage) + b'\r\n')


@app.route("/video_feed")
def video_feed():
    
    return Response(generate(),
                    mimetype="multipart/x-mixed-replace; boundary=frame")

In [8]:
def peopleDetector():

    global vs, outputFrame, lock

    while True:
        frame = vs.read()
        
        if (frame[0]):
            frame = frame[1]

             # detect people in the image
            (rects, weights) = hog.detectMultiScale(frame, winStride=(4, 4), 
                                                    padding=(8, 8), scale=1.05)

            if rects is not None:
                
            # apply non-maxima suppression to the bounding boxes using a
            # fairly large overlap threshold to try to maintain overlapping
            # boxes that are still people
            
                rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
                pick = non_max_suppression(rects, probs=None, overlapThresh=0.65)
                # draw the final bounding boxes
                for (xA, yA, xB, yB) in pick:
                    cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 255, 0), 2)          
                    
            with lock:
                outputFrame = frame.copy()

In [ ]:
t = threading.Thread(target=peopleDetector)
t.daemon = True
t.start()
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Jun/2020 12:55:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2020 12:55:51] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2020 12:59:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2020 13:00:35] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2020 13:00:41] "GET /video_feed HTTP/1.1" 200 -
